In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('pr.csv')
df



,text,target
0,great story anyone understand power one libera...,1
1,tirana reuters albania planning try lure five ...,0
2,washington reuters u president donald trump sa...,0
3,recently noticed seen melania trump recently l...,1
4,washington reuters u president elect donald tr...,0
...,...,...
44893,washington reuters california wisconsin said u...,0
44894,reuters gift wrapped package addressed u treas...,0
44895,new york reuters continental resource chief ex...,0
44896,wow talk clueless austen fletcher approach ant...,1


In [2]:
df.isna().sum()

text      632
target      0
dtype: int64

In [3]:
df.dropna(inplace=True)

In [4]:
df.isna().sum()

text      0
target    0
dtype: int64

In [5]:
text=df.text

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec=TfidfVectorizer()
data=vec.fit_transform(text)
data

<44266x115519 sparse matrix of type '<class 'numpy.float64'>'
	with 7282168 stored elements in Compressed Sparse Row format>

In [7]:
print(data)

  (0, 110645)	0.025633113050289265
  (0, 98893)	0.07041422728908041
  (0, 26154)	0.034047130889630274
  (0, 73222)	0.029636289679714788
  (0, 23216)	0.06955463286668621
  (0, 38818)	0.03263131686918805
  (0, 86487)	0.056151272899188746
  (0, 28538)	0.055476159801884126
  (0, 79944)	0.020505382454013465
  (0, 12258)	0.05643312174282786
  (0, 40084)	0.026213099396897605
  (0, 98964)	0.04036368455694733
  (0, 49079)	0.053367902337204016
  (0, 71225)	0.04434068630286021
  (0, 60419)	0.028711999850244963
  (0, 12370)	0.037295415381100344
  (0, 48419)	0.03081227588913007
  (0, 38798)	0.060417644532626144
  (0, 46756)	0.08293766332145074
  (0, 39059)	0.06884220338745133
  (0, 28776)	0.05938748267657344
  (0, 87388)	0.04294959317059809
  (0, 28843)	0.03794067703199279
  (0, 98982)	0.06964695432460498
  (0, 53459)	0.0580580602706257
  :	:
  (44265, 26154)	0.033723667928875704
  (44265, 79944)	0.06093171646757109
  (44265, 60419)	0.028439223018891648
  (44265, 48419)	0.0152597727483464
  (44265,

In [8]:
y=df['target']

In [10]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow as tf
from scipy.sparse import csr_matrix

x_train, x_test, y_train, y_test = train_test_split(data, y, test_size=0.3, random_state=42)

# Scale the data using MaxAbsScaler
scaler = MaxAbsScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Convert the scaled data back to sparse matrices
x_train_csr = csr_matrix(x_train_scaled)
x_test_csr = csr_matrix(x_test_scaled)

# Convert labels to numpy array
y_train = np.array(y_train)
y_test = np.array(y_test)
# Define a generator to yield batches of data
def data_generator(x_data, y_data, batch_size):
    num_samples = x_data.shape[0]
    while True:
        for offset in range(0, num_samples, batch_size):
            batch_x = x_data[offset:offset+batch_size].toarray()
            batch_y = y_data[offset:offset+batch_size]
            yield batch_x, batch_y
# Define a simpler ANN model
model = Sequential()
model.add(Dense(64, input_shape=(x_train_scaled.shape[1],), activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early stopping and learning rate reduction callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=0.00001)

# Training parameters
batch_size = 32
steps_per_epoch = x_train_csr.shape[0] // batch_size
validation_steps = x_test_csr.shape[0] // batch_size

# Train the model
history = model.fit(
    data_generator(x_train_csr, y_train, batch_size),
    steps_per_epoch=steps_per_epoch,
    epochs=100,
    validation_data=data_generator(x_test_csr, y_test, batch_size),
    validation_steps=validation_steps,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluate the model
score = model.evaluate(data_generator(x_test_csr, y_test, batch_size), steps=validation_steps)
print('Test accuracy:', score[1])

# Save the model to a file
model.save('fake_news_detection_model_simple.h5')


Epoch 1/100
968/968 ━━━━━━━━━━━━━━━━━━━━ 136s 139ms/step - accuracy: 0.9596 - loss: 0.1779 - val_accuracy: 0.9883 - val_loss: 0.0340 - learning_rate: 0.0010
Epoch 2/100
968/968 ━━━━━━━━━━━━━━━━━━━━ 138s 142ms/step - accuracy: 0.9982 - loss: 0.0072 - val_accuracy: 0.9902 - val_loss: 0.0311 - learning_rate: 0.0010
Epoch 3/100
968/968 ━━━━━━━━━━━━━━━━━━━━ 133s 138ms/step - accuracy: 0.9999 - loss: 0.0011 - val_accuracy: 0.9900 - val_loss: 0.0312 - learning_rate: 0.0010
Epoch 4/100
968/968 ━━━━━━━━━━━━━━━━━━━━ 132s 136ms/step - accuracy: 0.9999 - loss: 7.6083e-04 - val_accuracy: 0.9901 - val_loss: 0.0319 - learning_rate: 0.0010
Epoch 5/100
968/968 ━━━━━━━━━━━━━━━━━━━━ 134s 138ms/step - accuracy: 0.9999 - loss: 6.7931e-04 - val_accuracy: 0.9901 - val_loss: 0.0326 - learning_rate: 0.0010
Epoch 6/100
968/968 ━━━━━━━━━━━━━━━━━━━━ 137s 142ms/step - accuracy: 1.0000 - loss: 1.3885e-04 - val_accuracy: 0.9904 - val_loss: 0.0328 - learning_rate: 5.0000e-04
Epoch 7/100
968/968 ━━━━━━━━━━━━━━━━━━━━ 1

Test accuracy: 0.9902108311653137


In [12]:
import joblib

# Save the vectorizer and scaler
joblib.dump(vec, 'tfidf_vectorizer.joblib')
joblib.dump(scaler, 'maxabs_scaler.joblib')


['maxabs_scaler.joblib']

In [11]:

model.summary()



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 64)             │     7,393,280 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,180,037 (84.61 MB)

 Trainable params: 7,393,345 (28.20 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 14,786,692 (56.41 MB)

In [13]:
from sklearn.metrics import confusion_matrix, classification_report

# Assuming model is your trained Keras model
# data_generator generates batches of data (X, y)

# Evaluate the model
predictions = model.predict(data_generator(x_test_csr, y_test, batch_size), steps=validation_steps)
y_pred = np.where(predictions > 0.5, 1, 0)

# Compute confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)

# Print confusion matrix
print("Confusion Matrix:")
print(conf_mat)

# Compute classification report
class_report = classification_report(y_test, y_pred)

# Print classification report
print("Classification Report:")
print(class_report)


415/415 ━━━━━━━━━━━━━━━━━━━━ 14s 33ms/step
Confusion Matrix:
[[6367   45]
 [  85 6783]]
Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      6412
           1       0.99      0.99      0.99      6868

    accuracy                           0.99     13280
   macro avg       0.99      0.99      0.99     13280
weighted avg       0.99      0.99      0.99     13280

